In [1]:
txt_best_grids = "best_results_cup_nn.txt"
grid_results_name = 'grid_results_cup_nn_v6.csv'

In [2]:
# !pip install ipython-autotime
# !pip install -U scikit-learn
%load_ext autotime

time: 0 ns (started: 2021-01-23 11:18:01 +01:00)


In [3]:
!python --version

Python 3.8.6
time: 15 ms (started: 2021-01-23 11:18:01 +01:00)


In [4]:
import os

os.chdir('../../')
os.getcwd()

'C:\\Users\\Luca\\Desktop\\Elle\\University\\0 - UNIPI\\ML - Machine Learning\\Workspace\\ml-project'

time: 0 ns (started: 2021-01-23 11:18:01 +01:00)


In [5]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

from src.utility import *

time: 2.66 s (started: 2021-01-23 11:18:01 +01:00)


In [6]:
import keras.backend as K


def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

time: 31 ms (started: 2021-01-23 11:18:03 +01:00)


# Import Dataset

In [7]:
def load_cup():
    ml_cup = np.delete(np.genfromtxt('cup/ML-CUP20-TR.csv', 
                                     delimiter=','), obj=0, axis=1)
    return ml_cup[:, :-2], ml_cup[:, -2:]

def load_cup_blind():
    return np.delete(np.genfromtxt('cup/ML-CUP20-TS.csv',
                                   delimiter=','), obj=0, axis=1)
    
def mean_euclidean_error(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    return np.mean(np.linalg.norm(y_pred - y_true, axis=1))

time: 0 ns (started: 2021-01-23 11:18:03 +01:00)


In [8]:
X, y = load_cup()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

time: 16 ms (started: 2021-01-23 11:18:03 +01:00)


In [9]:
print(X.shape)
print(y.shape)

(1524, 10)
(1524, 2)
time: 0 ns (started: 2021-01-23 11:18:03 +01:00)


In [10]:
# data = np.genfromtxt('ML-CUP20-TR.csv', delimiter=',', dtype=np.float64)
# X = data[:, 1:-2]
# y = data[:, -2:]

# print(X.shape)
# print(y.shape)

time: 0 ns (started: 2021-01-23 11:18:03 +01:00)


In [11]:
# Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.20, random_state=42)
# print(Xtrain.shape)
# print(ytrain.shape)
# print(Xval.shape)
# print(yval.shape)

time: 0 ns (started: 2021-01-23 11:18:03 +01:00)


# Grid Search - Gradient Descent

In [12]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import L2

# Function to create model, required for KerasRegressor
def create_model(learn_rate=0.01, momentum=0, neurons_layer_1=10,
                 init_mode='uniform', activation='relu',  weight_decay = 0.0,
                 nesterov = False, neurons_layer_2 = 5):
  # create model
    model = Sequential()
    model.add(layers.InputLayer(input_shape=(10)))
    model.add(Dense(neurons_layer_1, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(neurons_layer_2, activation=activation,
                    kernel_initializer=init_mode, kernel_regularizer= L2(weight_decay)))
    model.add(Dense(2))
    optimizer = SGD(
      learning_rate=learn_rate, momentum=momentum, nesterov=nesterov)
    # Compile model
    model.compile(optimizer=optimizer,
              loss=euclidean_distance_loss)
    return model

BATCH_SIZE = len(X_train)

# create model
model = KerasRegressor(build_fn=create_model, verbose=2) #verbose = 0
# define the grid search parameters
batch_size = [64, 128]
epochs = [1000]
learn_rate = [0.006, 0.009, 0.01, 0.03]
momentum = [0.8, 0.85, 0.9, 0.95]
nesterov = [True]
neurons_layer_1 = [130]
neurons_layer_2 = [30]
init_mode = ['glorot_uniform']
activation = ['sigmoid']
weight_decay = [0.00001, 0.00009, 0.0001, 0.0003]

param_grid = dict(batch_size=batch_size, epochs=epochs, learn_rate=learn_rate,
                  momentum=momentum, neurons_layer_1=neurons_layer_1, 
                  init_mode=init_mode, activation=activation, weight_decay= weight_decay,
                  nesterov= nesterov, neurons_layer_2= neurons_layer_2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs= 6, cv=3, return_train_score=True, verbose=True)
grid_result = grid.fit(X_train, y_train)


Fitting 3 folds for each of 128 candidates, totalling 384 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  2.2min
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:  9.8min
[Parallel(n_jobs=6)]: Done 384 out of 384 | elapsed: 16.4min finished


Epoch 1/1000
20/20 - 0s - loss: 51.5644
Epoch 2/1000
20/20 - 0s - loss: 34.9242
Epoch 3/1000
20/20 - 0s - loss: 21.0208
Epoch 4/1000
20/20 - 0s - loss: 17.2681
Epoch 5/1000
20/20 - 0s - loss: 16.5136
Epoch 6/1000
20/20 - 0s - loss: 16.2250
Epoch 7/1000
20/20 - 0s - loss: 15.5040
Epoch 8/1000
20/20 - 0s - loss: 13.2241
Epoch 9/1000
20/20 - 0s - loss: 10.0882
Epoch 10/1000
20/20 - 0s - loss: 8.8583
Epoch 11/1000
20/20 - 0s - loss: 8.6395
Epoch 12/1000
20/20 - 0s - loss: 8.5687
Epoch 13/1000
20/20 - 0s - loss: 8.5094
Epoch 14/1000
20/20 - 0s - loss: 8.4688
Epoch 15/1000
20/20 - 0s - loss: 8.3339
Epoch 16/1000
20/20 - 0s - loss: 8.2285
Epoch 17/1000
20/20 - 0s - loss: 8.1221
Epoch 18/1000
20/20 - 0s - loss: 7.9378
Epoch 19/1000
20/20 - 0s - loss: 7.7813
Epoch 20/1000
20/20 - 0s - loss: 7.5746
Epoch 21/1000
20/20 - 0s - loss: 7.3507
Epoch 22/1000
20/20 - 0s - loss: 7.1219
Epoch 23/1000
20/20 - 0s - loss: 6.8996
Epoch 24/1000
20/20 - 0s - loss: 6.6292
Epoch 25/1000
20/20 - 0s - loss: 6.3726


Epoch 204/1000
20/20 - 0s - loss: 2.7332
Epoch 205/1000
20/20 - 0s - loss: 2.6957
Epoch 206/1000
20/20 - 0s - loss: 2.7256
Epoch 207/1000
20/20 - 0s - loss: 2.6841
Epoch 208/1000
20/20 - 0s - loss: 2.7062
Epoch 209/1000
20/20 - 0s - loss: 2.6763
Epoch 210/1000
20/20 - 0s - loss: 2.7394
Epoch 211/1000
20/20 - 0s - loss: 2.6768
Epoch 212/1000
20/20 - 0s - loss: 2.6953
Epoch 213/1000
20/20 - 0s - loss: 2.6824
Epoch 214/1000
20/20 - 0s - loss: 2.6969
Epoch 215/1000
20/20 - 0s - loss: 2.6721
Epoch 216/1000
20/20 - 0s - loss: 2.6907
Epoch 217/1000
20/20 - 0s - loss: 2.6659
Epoch 218/1000
20/20 - 0s - loss: 2.6609
Epoch 219/1000
20/20 - 0s - loss: 2.6705
Epoch 220/1000
20/20 - 0s - loss: 2.6784
Epoch 221/1000
20/20 - 0s - loss: 2.6788
Epoch 222/1000
20/20 - 0s - loss: 2.6603
Epoch 223/1000
20/20 - 0s - loss: 2.7004
Epoch 224/1000
20/20 - 0s - loss: 2.6572
Epoch 225/1000
20/20 - 0s - loss: 2.6876
Epoch 226/1000
20/20 - 0s - loss: 2.6888
Epoch 227/1000
20/20 - 0s - loss: 2.6599
Epoch 228/1000
2

Epoch 404/1000
20/20 - 0s - loss: 2.5348
Epoch 405/1000
20/20 - 0s - loss: 2.5258
Epoch 406/1000
20/20 - 0s - loss: 2.5186
Epoch 407/1000
20/20 - 0s - loss: 2.5138
Epoch 408/1000
20/20 - 0s - loss: 2.5234
Epoch 409/1000
20/20 - 0s - loss: 2.5116
Epoch 410/1000
20/20 - 0s - loss: 2.5684
Epoch 411/1000
20/20 - 0s - loss: 2.4993
Epoch 412/1000
20/20 - 0s - loss: 2.5193
Epoch 413/1000
20/20 - 0s - loss: 2.5090
Epoch 414/1000
20/20 - 0s - loss: 2.5033
Epoch 415/1000
20/20 - 0s - loss: 2.5108
Epoch 416/1000
20/20 - 0s - loss: 2.5089
Epoch 417/1000
20/20 - 0s - loss: 2.5245
Epoch 418/1000
20/20 - 0s - loss: 2.5098
Epoch 419/1000
20/20 - 0s - loss: 2.5421
Epoch 420/1000
20/20 - 0s - loss: 2.5050
Epoch 421/1000
20/20 - 0s - loss: 2.5236
Epoch 422/1000
20/20 - 0s - loss: 2.5328
Epoch 423/1000
20/20 - 0s - loss: 2.4987
Epoch 424/1000
20/20 - 0s - loss: 2.5257
Epoch 425/1000
20/20 - 0s - loss: 2.4915
Epoch 426/1000
20/20 - 0s - loss: 2.5017
Epoch 427/1000
20/20 - 0s - loss: 2.5125
Epoch 428/1000
2

Epoch 604/1000
20/20 - 0s - loss: 2.4072
Epoch 605/1000
20/20 - 0s - loss: 2.4360
Epoch 606/1000
20/20 - 0s - loss: 2.3992
Epoch 607/1000
20/20 - 0s - loss: 2.4049
Epoch 608/1000
20/20 - 0s - loss: 2.4150
Epoch 609/1000
20/20 - 0s - loss: 2.4028
Epoch 610/1000
20/20 - 0s - loss: 2.4154
Epoch 611/1000
20/20 - 0s - loss: 2.4058
Epoch 612/1000
20/20 - 0s - loss: 2.4205
Epoch 613/1000
20/20 - 0s - loss: 2.4253
Epoch 614/1000
20/20 - 0s - loss: 2.4013
Epoch 615/1000
20/20 - 0s - loss: 2.4267
Epoch 616/1000
20/20 - 0s - loss: 2.4027
Epoch 617/1000
20/20 - 0s - loss: 2.4049
Epoch 618/1000
20/20 - 0s - loss: 2.3999
Epoch 619/1000
20/20 - 0s - loss: 2.4332
Epoch 620/1000
20/20 - 0s - loss: 2.4012
Epoch 621/1000
20/20 - 0s - loss: 2.3871
Epoch 622/1000
20/20 - 0s - loss: 2.3849
Epoch 623/1000
20/20 - 0s - loss: 2.4077
Epoch 624/1000
20/20 - 0s - loss: 2.4065
Epoch 625/1000
20/20 - 0s - loss: 2.4071
Epoch 626/1000
20/20 - 0s - loss: 2.4234
Epoch 627/1000
20/20 - 0s - loss: 2.3999
Epoch 628/1000
2

Epoch 804/1000
20/20 - 0s - loss: 2.3137
Epoch 805/1000
20/20 - 0s - loss: 2.3271
Epoch 806/1000
20/20 - 0s - loss: 2.3200
Epoch 807/1000
20/20 - 0s - loss: 2.3346
Epoch 808/1000
20/20 - 0s - loss: 2.3258
Epoch 809/1000
20/20 - 0s - loss: 2.3147
Epoch 810/1000
20/20 - 0s - loss: 2.3262
Epoch 811/1000
20/20 - 0s - loss: 2.3247
Epoch 812/1000
20/20 - 0s - loss: 2.3279
Epoch 813/1000
20/20 - 0s - loss: 2.3313
Epoch 814/1000
20/20 - 0s - loss: 2.3220
Epoch 815/1000
20/20 - 0s - loss: 2.3228
Epoch 816/1000
20/20 - 0s - loss: 2.3255
Epoch 817/1000
20/20 - 0s - loss: 2.3267
Epoch 818/1000
20/20 - 0s - loss: 2.3386
Epoch 819/1000
20/20 - 0s - loss: 2.3156
Epoch 820/1000
20/20 - 0s - loss: 2.3244
Epoch 821/1000
20/20 - 0s - loss: 2.3214
Epoch 822/1000
20/20 - 0s - loss: 2.3195
Epoch 823/1000
20/20 - 0s - loss: 2.3511
Epoch 824/1000
20/20 - 0s - loss: 2.3272
Epoch 825/1000
20/20 - 0s - loss: 2.3217
Epoch 826/1000
20/20 - 0s - loss: 2.3169
Epoch 827/1000
20/20 - 0s - loss: 2.3160
Epoch 828/1000
2

In [13]:
import json

grid_results_name_str = f'Results wrt: {grid_results_name} \n'
grid_params_str = f'Grid on: {json.dumps(grid_result.param_grid)} \n'
grid_results_str = f'Best: {grid_result.best_score_} using {grid_result.best_params_} \n'

with open(txt_best_grids, "a") as file_object:
    file_object.write(grid_results_name_str)
    file_object.write(grid_params_str)
    file_object.write(grid_results_str)
    file_object.write('\n')

time: 0 ns (started: 2021-01-23 11:34:38 +01:00)


In [14]:
print(grid_results_str)

Best: -2.79458220799764 using {'activation': 'sigmoid', 'batch_size': 64, 'epochs': 1000, 'init_mode': 'glorot_uniform', 'learn_rate': 0.01, 'momentum': 0.8, 'nesterov': True, 'neurons_layer_1': 130, 'neurons_layer_2': 30, 'weight_decay': 1e-05} 

time: 0 ns (started: 2021-01-23 11:34:38 +01:00)


In [15]:
import pandas as pd


df = pd.DataFrame(grid.cv_results_)[['rank_test_score','mean_test_score',
                                     'std_test_score','mean_train_score',
                                     'std_train_score','param_activation',
                                     'param_batch_size','param_epochs',
                                     'param_init_mode', 'param_learn_rate',
                                     'param_momentum','param_nesterov',
                                     'param_neurons_layer_1', 'param_neurons_layer_2',
                                     'param_weight_decay','mean_fit_time']].sort_values(by='rank_test_score')

df.rename(columns={'param_activation': 'activation',
                   'param_batch_size': 'batch_size',
                   'param_epochs': 'epochs',
                   'param_init_mode': 'init_mode',
                   'param_learn_rate': 'learn_rate',
                   'param_momentum': 'momentum',
                   'param_init_mode': 'init_mode',
                   'param_weight_decay': 'weight_decay',
                   'param_nesterov': 'nesterov',
                   'param_neurons_layer_1': 'neurons_layer_1',
                   'param_neurons_layer_2': 'neurons_layer_2',
                   'mean_test_score': 'mean_val_score',
                   'std_test_score': 'std_val_score',                
                   'rank_test_score': 'rank_val_score'}, inplace=True)

df.mean_train_score *= -1
df.mean_val_score *= -1
df

,rank_val_score,mean_val_score,std_val_score,mean_train_score,std_train_score,activation,batch_size,epochs,init_mode,learn_rate,momentum,nesterov,neurons_layer_1,neurons_layer_2,weight_decay,mean_fit_time
32,1,2.794582,0.034712,2.217048,0.034607,sigmoid,64,1000,glorot_uniform,0.01,0.8,True,130,30,1e-05,17.424402
96,2,2.813667,0.034752,2.445883,0.042446,sigmoid,128,1000,glorot_uniform,0.01,0.8,True,130,30,1e-05,10.935316
100,3,2.814591,0.020436,2.338923,0.026214,sigmoid,128,1000,glorot_uniform,0.01,0.85,True,130,30,1e-05,10.961610
88,4,2.814980,0.028322,2.232693,0.051617,sigmoid,128,1000,glorot_uniform,0.009,0.9,True,130,30,1e-05,11.272960
104,5,2.814996,0.040632,2.204568,0.049686,sigmoid,128,1000,glorot_uniform,0.01,0.9,True,130,30,1e-05,11.117527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,124,3.741744,0.116931,1.719074,0.045157,sigmoid,64,1000,glorot_uniform,0.03,0.95,True,130,30,0.0003,17.307510
124,125,3.763168,0.196919,1.439841,0.145478,sigmoid,128,1000,glorot_uniform,0.03,0.95,True,130,30,1e-05,11.175594
61,126,3.893317,0.020259,1.327377,0.027801,sigmoid,64,1000,glorot_uniform,0.03,0.95,True,130,30,9e-05,17.488603
62,127,3.961533,0.165504,1.356387,0.142382,sigmoid,64,1000,glorot_uniform,0.03,0.95,True,130,30,0.0001,18.059354


time: 16 ms (started: 2021-01-23 11:34:38 +01:00)


In [16]:
df.to_csv(grid_results_name, index= False)

time: 0 ns (started: 2021-01-23 11:34:38 +01:00)


In [17]:
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# import csv

# with open(single_grid_results_filename, mode='w') as grid:
#     grid = csv.writer(grid, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#     for mean, stdev, param in zip(means, stds, params):
#       grid.writerow([mean, stdev, param])

time: 0 ns (started: 2021-01-23 11:34:38 +01:00)


In [18]:
# from sklearn.externals import joblib

# # #save your model or results
# # joblib.dump(grid_result, 'gs_object_monk3.pkl')
# # #load your model for further usage
# # boh = joblib.load("gs_object_monk3.pkl")

# # joblib.dump(grid.best_estimator_, 'gs_best_estimator_monk3.pkl', compress = 1)
# # filename = 'finalized_model.sav'
# # pickle.dump(model, open(filename, 'wb'))

time: 0 ns (started: 2021-01-23 11:34:38 +01:00)
